<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# Reddit and HuggingFace Starter Kit

In this live coding session, we leverage the Python Reddit API Wrapper (`PRAW`) to retrieve data from subreddits on [Reddit](https://www.reddit.com), and perform sentiment analysis using [`pipelines`](https://huggingface.co/docs/transformers/main_classes/pipelines) from [HuggingFace ( 🤗 the GitHub of Machine Learning )](https://techcrunch.com/2022/05/09/hugging-face-reaches-2-billion-valuation-to-build-the-github-of-machine-learning/), powered by [transformer](https://arxiv.org/pdf/1706.03762.pdf).

## Objectives

At the end of the session, you will 

- know how to work with APIs
- feel more comfortable navigating thru documentation, even inspecting the source code
- understand what a `pipeline` object is in HuggingFace
- perform sentiment analysis using `pipeline`
- run a python script in command line and get the results

## How to Submit

- At the end of each task, commit* the work into your own remote repo
- After completing all three tasks, make sure to push the notebook containing all code blocks and output cells to your remote repo
- Submit the link to the notebook in Canvas

\***NEVER** commit a notebook displaying errors unless it is instructed otherwise. However, commit often; recall git ABC = **A**lways **B**e **C**ommitting.

## Tasks

### Task I: Instantiate a Reddit API Object

The first task is to instantiate a Reddit API object using [PRAW](https://praw.readthedocs.io/en/stable/), through which you will retrieve data. PRAW is a wrapper for [Reddit API](https://www.reddit.com/dev/api) that makes interacting with the Reddit API easier unless you are already an expert of [`requests`](https://docs.python-requests.org/en/latest/).

#### 0.  Get updates from `FourthBrain/MLE-7`

Under your forked local repo, **fetch** and download new updates from repo `FourthBrain/MLE-7` locally so you can start development. 
    
<details>
<summary>If you haven't added `FourthBrain/MLE-7` as a remote repo, click here for instructions:</summary>   
You fork repo `FourthBrain/MLE-7` to `yourhandle/MLE-7`, clone it locally, and now you are under directory `MLE-7`. By default, you will see one server name `origin` pointing to your repo:  
    
```
$git remote -v 
origin  git@github.com:yourhandle/MLE-7.git (fetch)
origin  git@github.com:yourhandle/MLE-7.git (push)
```

Think of fetch = read and push = write. 

Now add `FourthBrain/MLE-7` as a remote repo

```
$git add remote fourthbrain git@github.com:FourthBrain/MLE-7.git
$git remote -v
fourthbrain	git@github.com:FourthBrain/MLE-7.git (fetch)
fourthbrain	git@github.com:FourthBrain/MLE-7.git (push)
origin git@github.com:yourhandle/MLE-7.git (fetch)
origin git@github.com:yourhandle/MLE-7.git (push)
```

then before each session starts, run `git fetch fourthbrain` to get updates (why not `git pull`?).

check out [Working with Remotes](https://git-scm.com/book/en/v2/Git-Basics-Working-with-Remotes) for more explanations.
</details>

#### 1. Install packages

```
conda activate {your_virtual_environment_name}
pip install -U transformers praw torch numpy pandas
```

####  2. Create a new app on Reddit 

Create a new app on Reddit and save secret tokens; refer to [post in medium](https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c) for more details.

- Create a Reddit account if you don't have one, log into your account.
- To access the API, we need create an app. Slight updates, on the website, you need to navigate to `preference` > `app`, or click [this link](https://www.reddit.com/prefs/apps) and scroll all the way down. 
- Click to create a new app, fill in the **name**, choose `script`, fill in  **description** and **redirect url** ( The redirect URI is where the user is sent after they've granted OAuth access to your application; more info [here](and details are in [](https://github.com/reddit-archive/reddit/wiki/OAuth2) for our purpose, you can enter some random url, e.g., www.google.com; ) as shown below.
    <img src="https://miro.medium.com/max/700/1*lRBvxpIe8J2nZYJ6ucMgHA.png" width="500"/>
- Jolt down `client_id` (left upper corner) and `client_secret` 

    NOTE: CLIENT_ID refers to 'personal use script" and CLIENT_SECRET to secret.
    <div>
    <img src="https://miro.medium.com/max/700/1*7cGAKth1PMrEf2sHcQWPoA.png" width="300"/>
    </div>

- Create `secrets.py` in the same directory with this notebook, fill in `client_id` and `secret_id` obtained from the last step. We will need to import those constants in the next step.
    ```
    REDDIT_API_CLIENT_ID = {client_id}
    REDDIT_API_CLIENT_SECRET = {secret_id}
    REDDIT_API_USER_AGENT = {can_be_any_string, e.g., : "BotBot"}
    ```
- Add `secrets.py` to your `.gitignore` file if not already done. NEVER push credentials to a repo, private or public. 

#### 3. Instantiate a `Reddit` object

Now you are ready to create a read-only `Reddit` instance. Refer to [documentation](https://praw.readthedocs.io/en/stable/code_overview/reddit_instance.html) when necessary.

In [8]:
import praw
import secrets
from praw.models import MoreComments
import random

# Create a Reddit object which allows us to interact with the Reddit API
reddit = praw.Reddit(
        client_id=secrets.REDDIT_API_CLIENT_ID,
        client_secret=secrets.REDDIT_API_CLIENT_SECRET,
        user_agent=secrets.REDDIT_API_USER_AGENT
)

In [3]:
print(reddit) 

<details>
<summary>Expected output:</summary>   

```<praw.reddit.Reddit object at 0x10f8a0ac0>```
</details>

#### 4. Instantiate a `subreddit` object

Lastly, create a `subreddit` object for your favorite subreddit and inspect the object. The expected output you will see ar from `r/machinelearning` unless otherwise specified.

In [22]:
subreddit = reddit.subreddit("machinelearning") 

What is the display name of the subreddit?

In [23]:
subreddit

Subreddit(display_name='machinelearning')

<details>
<summary>Expected output:</summary>   

    machinelearning
</details>

How about its title, is it different from the display name?

In [24]:
subreddit.title

'Machine Learning'

<details>
<summary>Expected output:</summary>   

    Machine Learning
</details>

Print out the description of the subreddit:

In [25]:
print(subreddit.description)

**[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
--------
+[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
--------
+[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
--------
+[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
--------
+[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ANews)
--------
***[@slashML on Twitter](https://twitter.com/slashML)***
--------
***[Chat with us on Slack](https://join.slack.com/t/rml-talk/shared_invite/enQtNjkyMzI3NjA2NTY2LWY0ZmRjZjNhYjI5NzYwM2Y0YzZhZWNiODQ3ZGFjYmI2NTU3YjE1ZDU5MzM2ZTQ4ZGJmOTFmNWVkMzFiMzVhYjg)***
--------
**Beginners:**
--------
Please have a look at [our FAQ and Link-Collection](http://www.reddit.com/r/MachineLearning/wiki/index)

[Metacademy](http://www.metacademy.org) is a great resource which compiles le

<details>
<summary>Expected output:</summary>

    **[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
    --------
    +[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
    --------
    +[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
    --------
    +[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
    --------
    +[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict
</details>

### Task II: Parse comments

#### 1. Top Posts of All Time

Find titles of top 10 posts of **all time** from your favorite subreddit. Refer to [Obtain Submission Instances from a Subreddit Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html)) if necessary. Verify if the titles match what you read on Reddit.

In [26]:
# try run this line, what do you see? press q once you are done
?subreddit.top 

Signature:
subreddit.top(
    *,
    time_filter: str = 'all',
    **generator_kwargs: Union[str, int, Dict[str, str]],
) -> Iterator[Any]
Docstring:
Return a :class:`.ListingGenerator` for top items.

:param time_filter: Can be one of: ``"all"``, ``"day"``, ``"hour"``,
    ``"month"``, ``"week"``, or ``"year"`` (default: ``"all"``).

:raises: :py:class:`ValueError` if ``time_filter`` is invalid.

Additional keyword arguments are passed in the initialization of
:class:`.ListingGenerator`.

This method can be used like:

.. code-block:: python

    reddit.domain("imgur.com").top(time_filter="week")
    reddit.multireddit(redditor="samuraisam", name="programming").top(time_filter="day")
    reddit.redditor("spez").top(time_filter="month")
    reddit.redditor("spez").comments.top(time_filter="year")
    reddit.redditor("spez").submissions.top(time_filter="all")
    reddit.subreddit("all").top(time_filter="hour")
File:      ~/anaconda3/envs/MLE7/lib/python3.9/site-packages/praw/models/list

In [32]:
submissions = [c for c in subreddit.top(limit=10) ]

In [37]:
submission = submissions[0]

In [40]:
submission.title

'[Project] From books to presentations in 10s with AR + ML'

In [46]:
for submission in submissions:
    print(submission.title)

[Project] From books to presentations in 10s with AR + ML
[D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
[R] First Order Motion Model applied to animate paintings
[N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
[D] This AI reveals how much time politicians stare at their phone at work
[D] Types of Machine Learning Papers
[D] The machine learning community has a toxicity problem
[Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
[P] Using oil portraits and First Order Model to bring the paintings back to life
[D] Convolution Neural Network Visualization - Made with Unity 3D and lots of Code / source - stefsietz (IG)


<details> <summary>Expected output:</summary>

    [Project] From books to presentations in 10s with AR + ML
    [D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
    [R] First Order Motion Model applied to animate paintings
    [N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
    [D] This AI reveals how much time politicians stare at their phone at work
    [D] Types of Machine Learning Papers
    [D] The machine learning community has a toxicity problem
    [Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
    [P] Using oil portraits and First Order Model to bring the paintings back to life
    [D] Convolution Neural Network Visualization - Made with Unity 3D and lots of Code / source - stefsietz (IG)    
</details>

#### 2. Top 10 Posts of This Week

What are the titles of the top 10 posts of **this week** from your favorite subreddit?

In [47]:
[sub.title for sub in subreddit.top(limit=10,time_filter="week")]

['[N] Ian Goodfellow, Apple’s director of machine learning, is leaving the company due to its return to work policy. In a note to staff, he said “I believe strongly that more flexibility would have been the best policy for my team.” He was likely the company’s most cited ML expert.',
 '[R][P] Thin-Plate Spline Motion Model for Image Animation + Gradio Web Demo',
 '[N] Hugging Face raised $100M at $2B to double down on community, open-source & ethics',
 "[P] I’ve been trying to understand the limits of some of the available machine learning models out there. Built an app that lets you try a mix of CLIP from Open AI + Apple’s version of MobileNet, and more directly on your phone's camera roll.",
 '[R] RWKV-v2-RNN : A parallelizable RNN with transformer-level LM performance, and without using attention',
 '[P] T-SNE to view and order your Spotify tracks',
 '[D] Are there any software engineers that switched into a machine learning role and found it a lot more stressful due to deadlines co

<details><summary>Expected output:</summary>

    [N] Ian Goodfellow, Apple’s director of machine learning, is leaving the company due to its return to work policy. In a note to staff, he said “I believe strongly that more flexibility would have been the best policy for my team.” He was likely the company’s most cited ML expert.
    [R][P] Thin-Plate Spline Motion Model for Image Animation + Gradio Web Demo
    [P] I’ve been trying to understand the limits of some of the available machine learning models out there. Built an app that lets you try a mix of CLIP from Open AI + Apple’s version of MobileNet, and more directly on your phone's camera roll.
    [R] Meta is releasing a 175B parameter language model
    [N] Hugging Face raised $100M at $2B to double down on community, open-source & ethics
    [P] T-SNE to view and order your Spotify tracks
    [D] : HELP Finding a Book - A book written for Google Engineers about foundational Math to support ML
    [R] Scaled up CLIP-like model (~2B) shows 86% Zero-shot on Imagenet
    [D] Do you use NLTK or Spacy for text preprocessing?
    [D] Democratizing Diffusion Models - LDMs: High-Resolution Image Synthesis with Latent Diffusion Models, a 5-minute paper summary by Casual GAN Papers
</details>

#### 3. Comment Code

Add comments to the code block below to describe what each line of the code does (Refer to [Obtain Comment Instances Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html) when necessary). The code is adapted from [this tutorial](https://praw.readthedocs.io/en/stable/tutorials/comments.html)

The purpose is 
1. to understand what the code is doing 
2. start to comment your code whenever it is not self-explantory if you have not (others will thank you, YOU will thank you later 😊) 

In [48]:
%%time

# List to capture the top comments
top_comments = []
# extract reddit post (submission) - top 10 all time
for submission in subreddit.top(limit=10):
    # iteration over comments of the extracted post
    for top_level_comment in submission.comments:
        # Checking if it has more comments so we can do top_level_comment.comments
        # In this part we skip to append the comment given that it has the class MoreComments 
        if isinstance(top_level_comment,MoreComments):
            continue
        # Capture the top level comment
        top_comments.append(top_level_comment.body)

CPU times: user 1.1 s, sys: 27 ms, total: 1.13 s
Wall time: 1min 31s


#### 4. Inspect Comments

How many comments did you extract from the last step? Examine a few comments. 

In [49]:
# the answer may vary 693 for r/machinelearning
len(top_comments)

693

In [52]:
print([random.choice(top_comments) for _ in range(5)])

['As a biologist, this is most papers in that field too.', 'Smart move', 'Givin the ol Harry potter treatment', "> Fourthly, the way Yann LeCun talked about biases and fairness topics was insensitive. \n\nI understand why you might feel you have to say this, but it isn't true, and catering to that mindset is only going to provide a beachhead for future unreasonable backlashes. People who jumped on LeCun overplayed their hand, but they're still in the community, and will happily jump on other innocent remarks the second we let them think they've got a receptive audience for it. Saying that biased datasets cause problems is not a racist act, there are four lights.\n\n> People are becoming afraid to engage in fear of being called a racist or sexist, which in turn reinforces the diversity problem.\n\nVery big agree! We need to incentivize outreach and risk-taking.\n\n> \nSecondly, there is a reproducibility crisis. Tuning hyperparameters on the test set seem to be the standard practice now

<details> <summary>Some of the comments from `r/machinelearning` subreddit are:</summary>

    ['Awesome visualisation',
    'Similar to a stack or connected neurons.',
    'Will this Turing pass the Turing Test?']
</details>

#### 5. Extract Top Level Comment from Subreddit `TSLA`.

Write your code to extract top level comments from the top 10 topics of a time period, e.g., year, from subreddit `TSLA` and store them in a list `top_comments_tsla`.  

In [2]:
subreddit = reddit.subreddit("TSLA") 

In [5]:
top_comments_tsla = [comment.body for sub in subreddit.top(limit=10,time_filter="week") for comment in sub.comments  if not isinstance(comment,MoreComments)]

In [6]:
len(top_comments_tsla) # Expected: 174 for r/machinelearning

11

In [9]:
[random.choice(top_comments_tsla) for i in range(3)]

['If I ever get even I will get as far away from that crook as I can',
 'No gas needed to go from 740 to 1200 in no time!!',
 'This is the best tl;dr I could make, [original](https://www.reuters.com/technology/musk-says-44-billion-twitter-deal-hold-2022-05-13/) reduced by 90%. (I\'m a bot)\n*****\n> May 13 - Elon Musk tweeted on Friday that his $44-billion cash deal for Twitter Inc was &quot;Temporarily on hold&quot; while he waits for the social media company to provide data on the proportion of its fake accounts.\n\n> &quot;Twitter deal temporarily on hold pending details supporting calculation that spam/fake accounts do indeed represent less than 5% of users,&quot; Musk told his more than 92 million Twitter followers.\n\n> The language in the Twitter deal agreement, as in many recent mergers, does not allow Musk to walk away because of a deteriorating business environment, such as a drop in demand for advertising or because Twitter&#039;s shares have plunged.\n\n\n*****\n[**Extended

<details>
<summary>Some of the comments from `r/TSLA` subreddit:</summary>

    ['I bought puts',
    '100%',
    'Yes. And I’m bag holding 1200 calls for Friday and am close to throwing myself out the window']
</details>

### Task III: Sentiment Analysis

Let us analyze the sentiment of comments scraped from `r/TSLA` using a pre-trained HuggingFace model to make the inference. Take a [Quick tour](https://huggingface.co/docs/transformers/quicktour). 

#### 1. Import `pipeline`

In [10]:
from transformers import pipeline

#### 2. Create a Pipeline to Perform Task "sentiment-analysis"

In [11]:
sentiment_model = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

#### 3. Get one comment from list `top_comments_tsla` from Task II - 5.

In [16]:
comment = random.choice(top_comments_tsla)

In [17]:
comment

'If I ever get even I will get as far away from that crook as I can'

The example comment is: `'Bury Burry!!!!!'`. Print out what you get. For reproducibility, use the same comment in the next step; consider setting a seed.

#### 4. Make Inference!

In [20]:
#sentiment_model = pipeline("sentiment-analysis")
sentiment = sentiment_model(comment)
sentiment

[{'label': 'NEGATIVE', 'score': 0.9896872639656067}]

What is the type of the output `sentiment`?

```
For the comment the sentiment is Negative
```

In [22]:
print(f'The comment: {comment}')
print(f'Predicted Label is {sentiment[0]["label"]} and the score is {sentiment[0]["score"]:.3f}')

The comment: If I ever get even I will get as far away from that crook as I can
Predicted Label is NEGATIVE and the score is 0.990


For the example comment, the output is:

    The comment: Bury Burry!!!!!
    Predicted Label is NEGATIVE and the score is 0.989

### Task IV: Put All Together

Let's pull all the piece together, create a simple script that does 

- get the subreddit
- get comments from the top posts for given subreddit
- run sentiment analysis 

#### Complete the Script

Once you complete the code, running the following block writes the code into a new Python script and saves it as `top_tlsa_comment_sentiment.py` under the same directory with the notebook. 

In [23]:
%%writefile top_tlsa_comment_sentiment.py

import secrets
import random

from typing import Dict, List

from praw import Reddit
from praw.models.reddit.subreddit import Subreddit
from praw.models import MoreComments

from transformers import pipeline


def get_subreddit(display_name:str) -> Subreddit:
    """Get subreddit object from display name

    Args:
        display_name (str): [description]

    Returns:
        Subreddit: [description]
    """
    reddit = Reddit(
        client_id=secrets.REDDIT_API_CLIENT_ID,        
        client_secret=secrets.REDDIT_API_CLIENT_SECRET,
        user_agent=secrets.REDDIT_API_USER_AGENT
        )
    
    subreddit = reddit.subreddit(display_name) 
    return subreddit

def get_comments(subreddit:Subreddit, limit:int=3) -> List[str]:
    """ Get comments from subreddit

    Args:
        subreddit (Subreddit): [description]
        limit (int, optional): [description]. Defaults to 3.

    Returns:
        List[str]: List of comments
    """
    top_comments = []
    for submission in subreddit.top(limit=limit):
        for top_level_comment in submission.comments:
            if isinstance(top_level_comment, MoreComments):
                continue
            top_comments.append(top_level_comment.body)
    return top_comments

def run_sentiment_analysis(comment:str) -> Dict:
    """Run sentiment analysis on comment using default distilbert model
    
    Args:
        comment (str): [description]
        
    Returns:
        str: Sentiment analysis result
    """
    sentiment_model = pipeline("sentiment-analysis")
    sentiment = sentiment_model(comment)
    return sentiment[0]


if __name__ == '__main__':
    submission = get_subreddit("TSLA")
    comments = get_comments(submission)
    comment = random.choice(comments)
    sentiment = run_sentiment_analysis(comment)
    
    print(f'The comment: {comment}')
    print(f'Predicted Label is {sentiment["label"]} and the score is {sentiment["score"]:.3f}')

Overwriting top_tlsa_comment_sentiment.py


Run the following block to see the output.

In [24]:
!python top_tlsa_comment_sentiment.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
The comment: Lies...why would an R&D person in the company be involved in this? I don’t buy it.
Predicted Label is NEGATIVE and the score is 1.000


<details><summary> Expected output:</summary>

    No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
    The comment: When is DOGE flying
    Predicted Label is POSITIVE and the score is 0.689
</details>